# COMP90016 - Assignment 2



Version 3. Last edited 16/4/2020.

## Sample solutions

Well done on completing assignment 2! In general, students did a great job writing thoughtful written answers. The coding question was challenging, but many students produced correct translated ORFs to some if not all test cases.

We are pleased to say that only one or two students failed to upload a correctly formatted HTML file of their assignment along with their Juptyer notebook. This is a requirement for all future assignments. We will not be able to mark your assignment until you upload both files.

Here are some of the most common mistakes.
- Function output must be of the type specified in the instructions.
- In genetic code 21, ATA encodes methionine but not an initiation codon. This led to some incorrect ORFs being identified.
- The BLAST E-value is not a probability.
- tBLASTn is required when searching a nucleotide database with a protein query.
- Sensitivity is different to specificity. Sensitivity is not decreased if the number of false positive hits is increased.

The sample solutions below provide a guide for students to compare to their own answers. These solutions are not the most efficient or succinct; they are written to be simple and easy to understand.

## Semester 1, 2020¶

This assignment should be completed by each student individually. Make sure you read this entire document, and ask for help if anything is not clear. Any changes or clarification to this document will be announced via the LMS.

Please make sure you are aware of the University's rules on academic honesty and plagiarism, which are very strict: https://academichonesty.unimelb.edu.au/

Make sure you do not copy any code either from other students or from the internet. This is considered plagiarism. It is generally a good idea to avoid looking at any solutions as you may find it surprisingly difficult to generate your own solution to the problem once you have seen somebody else's.

Your completed notebook file containing all your answers will be turned in via LMS. No other files or formats will be accepted - **only upload the completed .ipynb file.**

## Overview

To complete the assignment you will need to finish the tasks in this notebook. There are multiple tasks that are connected in a logical order.

The tasks are a combination of writing your own code using library implementations, utilising bioinformatics tools and interpreting the results in short answer format.

>Word limits, where stated, will be strictly enforced! Answers exceeding the limit will **NOT** be marked.

In some cases, we have provided test input and test output that you can use to try out your solutions. These tests are just samples and are **not** exhaustive - they may warn you if you've made a mistake, but they are not guaranteed to. It's up to you to decide whether your code is correct.

## Marking

Cells that must be completed to receive marks are clearly labeled. Some cells are code cells, in which you must complete the code to solve a problem, and some are markdown cells, in which you must write your answers to short-answer questions. Please only add to graded cells, **do not remove or modify any text in graded cells** unless specified otherwise (please don't remove text like this: # ~~ GRADED CELL (1 mark) - complete this cell ~~ ).

No marks are allocated to commenting in your your code. Likewise, no marks are allocated to the speed or complexity of your code. We do however, encourage efficient and well commented code.

Your code will be tested on the comp90016_assignment_2_sequence.fasta file provided, as well as additional test cases. Your code must work on any correctly formatted FASTA file.

The total marks for the assignment add up to 100, and it will be worth 10% of your overall subject grade.

Part 1: 45 marks

Part 2: 55 marks

## Overview

In this assignment, you will use functions in the `skbio` library to write an open reading frame (ORF) finder. You may want to refer to sections of the `skbio` documentation for additional help (http://scikit-bio.org/docs/0.5.2). Additional to `skbio` and standard Python 3 functions and methods, you may also use any other libraries installed on the COMP90016 server including collections, numpy and pandas. These can be found using help ('modules').

You will also be asked to consider nested and overlapping ORFs and alternate genetic codes. In part 2, you will use BLAST to investigate a sequence from the genome of SARS-CoV-2 (COVID-19).

## Part 1 - Genetic codes and open reading frames
### Setup

We begin by importing the `skbio` library. The `skbio.io.read` function reads the *FASTA* file. The function returns a generator object which can be used to loop through the file. Read https://realpython.com/introduction-to-python-generators/ to better understand generators and iterators in Python (this is not necessary to complete the assignment).

The *FASTA* file you will be using can be downloaded from the LMS. Download the file and place it in the same folder where this notebook is. **DO NOT** rename the file. 

In [ ]:
# Import the skbio library.
import skbio

In [ ]:
# Read in the FASTA file to produce a generator object named registry.
fname = 'comp90016_assignment_2_sequence.fasta'
registry = skbio.io.read(fname, format = 'fasta')
registry

In [ ]:
# Assign the sequence to an skbio.DNA.sequence object named sequence.
for seq in registry:
    sequence = seq
sequence = skbio.sequence.DNA(seq, lowercase = True)
sequence

Up until this point in the COMP90016 course, we have assumed that the standard genetic code applies. However, some organisms use different variations of the genetic code. You can see the differences at the site below. Note that some genetic codes have multiple initiation codons, some are more commonly used than others. The first amino acid of any ORF  is always methionine, even when an alternative initiation codon is used. In any other position, that codon may encode a different amino acid but, as a start codon, it will always code for methionine.
>https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi

These genetic codes can be referred to by a number. For example, the standard genetic code is genetic code 1.
Fortunately, skbio has functions that can interpret alternate genetic codes. See the documentation for methods like .translate() and .translate_six_frames()

The GeneticCode function can also be used to help you incorporate the genetic codes into your functions.

In [ ]:
from skbio import GeneticCode

# Show the standard genetic code. Try changing the number to view a different code.
# Note that RNA codons are shown with the GeneticCode function where DNA codons are used on the NCBI site.
GeneticCode.from_ncbi(1)

Mitochondria are orgenelles that can be found within eukaryotic cells and are important for energy production. Mitochondria contain their own circular DNA genome that encodes a number of genes essential for oxidative phosphorylation. Mitochondria also maintain their own protein synthesis machinery and the genetic code for vertebrate mitochondria differs slightly from the standard genetic code. The genetic material of mitochondria is of interest to researchers as human diseases can result from mutations in mitochondrial DNA.

An open reading frame (ORF) is a continuous sequence of DNA that begins with a start codon and ends at a stop codon. ORFs can be located in any reading frame, can be overlapping or can be nested within another ORF.


### Questions
In the cells below, complete the following tasks:

Suppose you are studying a mitochondrial genome from a human patient. The mitochondiral DNA has a rearrangement compared to the reference mitochondrial genome. You suspect that this rearrangement might result in the creation of a novel ORF.

1.1. (5 marks, 50 words max) Describe the differences between the standard genetic code and the vertebrate mitochondrial genetic code.

1.2. (10 marks, 50 words max) What could be the consequences of using the standard genetic code when looking for ORFs in the mitochondrial genome of the patient?

1.3. (20 marks) Write a python function to identify the protein sequences resulting from the translation of the ORFs in a given DNA sequence. Return a list of skbio.sequence.Protein objects where the protein sequence is longer than min_len. Assume the input sequence is an skbio.sequence.DNA object. If the input is empty, return an empty list.
Use the genetic code encoded by genetic_code. If genetic_code is not one of the NCBI options, use the standard code. Assume ORFs can start with any of the initiation codons and finish with any of the stop codons specified in the genetic code. Assume min_len is an integer. Only consider ORFs wholly contained within the sequence (including the start and stop codons).

Consider the journal article below. Please read the summary and introduction. ORF-9b is an ORF from the genome of the SARS coronavirus that caused the SARS epidemic in 2003.

>https://www.cell.com/structure/fulltext/S0969-2126(06)00251-6?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS0969212606002516%3Fshowall%3Dtrue

1.4. (10 marks, 50 words max) Explain how a single DNA sequence could encode multiple, overlapping gene-products.

~~ GRADED CELL (5 marks) - complete this cell ~~

Describe the differences between the standard genetic code and the vertebrate mitochondrial genetic code. **50 words max**

TGA, ATA, AGA and AGG encode STOP, isoleucine, arginine and arginine in the standard code. The same codons encode tryptophan, methionine/START, STOP and STOP in the vertebrate mitochondrial code. Alternative initiation codons also differ between the codes.

~~ GRADED CELL (10 marks) - complete this cell ~~

What could be the consequences of using the standard genetic code when looking for ORFs in the mitochondrial genome of the patient? **50 words max**

The differences in start and stop codons between the genetic codes mean that true mitochondrial ORFs may be missed and false positive ORFs may be detected. Predicted ORFs may also be truncated or extended compared to true ORFs due to these differences.

In [ ]:
# ~~ GRADED CELL (20 marks) - complete this cell ~~
def translate_orfs(seq, min_len, genetic_code):
    """
    Translate the ORFs in all reading frames of a DNA sequence. 
    Assume seq is a skbio.sequence.DNA object.
    Assume min_len is an integer.
    Assume genetic_code is an integer.
    If genetic_code is not one of the NCBI options, use the standard code.
    Return a list of skbio.sequence.Protein objects.
    If the input sequence is empty, return an empty list.
    """
    
    # Initiate a list to contain the start codons for the genetic code.
    starts = []
    # Initiate a list to contain the two strands of the input sequence.
    strands = []
    # Initiate a list to contain potential ORF translations.
    untrimmed_translated_orfs = []
    # Initiate a list to contain the final translated ORFs.
    translated_orfs = []
    
    # If the input sequence is empty, return an empty list.
    if len(seq) == 0:
        return translated_orfs
    
    # Create a list of codes compatible with the GeneticCode function.
    compatible_codes = [1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 16, 21, 22, 23, 24, 25]
    # Set the genetic code to 1 if it not compatible with the GeneticCode function.
    if genetic_code not in compatible_codes:
        genetic_code = 1
    
    # Store the start codons for the genetic code in starts_ncbi.
    starts_ncbi = GeneticCode.from_ncbi(genetic_code)._start_codons
    # These start codons are encoded with numbers rather than nucleotide strings. This dictionary makes the conversion.
    nucleotide_dict = {0:'T',1:'C',2:'A',3:'G'}
    # Append the string of nucleotides for each start codon to starts.
    for i in range(len(starts_ncbi)):
        start_codon = ''
        for base in starts_ncbi[i]:
            start_codon += nucleotide_dict[base]
        starts.append(start_codon)
    
    # Store the forward and reverse strands of the input sequence in strands.
    strands.append(seq)
    strands.append(seq[::-1].complement())
    
    # Iterate through each strand and find substrings that begin with start codons.
    # Append the protein sequence of these substrings to untrimmed_orfs. 
    # Note that the protein sequences do not necessarily end with a *.
    for strand in strands:
        for i in range(len(strand)-3):
            if str(strand[i:i+3]) in starts:
                protein = str(strand[i:].translate(genetic_code))
                untrimmed_translated_orfs.append(protein)
    
    # Iterate through the untrimmed translated orfs and replace the first amino acid with a methionine.
    # Trim the translated ORFs at the first * and append the trimmed proteins to translated_orfs if longer than min_len.
    # Discard any translated ORFs that are not followed by a *.
    for orf in untrimmed_translated_orfs:
        orf = "M" + orf[1:]
        for i in range(len(orf)):
            if orf[i] == "*":
                if len(orf[:i]) > min_len:
                    translated_orfs.append(skbio.sequence.Protein(orf[:i]))
                break
    
    return translated_orfs


In [ ]:
# ~~ Test your function in this cell ~~
demo_sequence_a = skbio.sequence.DNA('GTTGGATTCATGAAAGA')
print(translate_orfs(demo_sequence_a, 2, 1)) # should give a single sequence: MDS
print(translate_orfs(demo_sequence_a, 2, 2)) # should give a single sequence: MHE
demo_sequence_b = skbio.sequence.DNA('ATGAAATGAATGTCTTGA')
print(translate_orfs(demo_sequence_b, 1, 1)) # should give two sequences: MK and MS
demo_sequence_c = skbio.sequence.DNA('ATGAAAATGTCTTGA')
print(translate_orfs(demo_sequence_c, 1, 100)) # should give two sequences: MKMS and MS

print(translate_orfs(sequence, 60, 21))

~~ GRADED CELL (10 marks) - complete this cell ~~

Explain how a single DNA sequence could encode multiple, overlapping gene-products. **50 words max**

There are several mechanisms by which a DNA sequence could encode multiple gene products. One example is that there could be two sets of overlapping start and stop codons in different reading frames.

## Part 2 - BLAST

### Setup
ORF-9b has been detected in multiple isolates of SARS coronavirus as well as in other coronaviruses that infect other animals. We would like to determine whether SARS-CoV-2 (COVID-19) has an ORF that is homologous to ORF-9b in other coronaviruses. We will be using BLAST on the command line for this purpose.

We will be using a sequence from an isolate of SARS-CoV-2 sequenced in December 2019 in Wuhan, China. 

>https://www.viprbrc.org/brc/viprStrainDetails.spg?ncbiAccession=MN908947&decorator=corona

Using the ViPR link above, navigate to and download a FASTA file of the DNA sequence for the nucleocapsid (N) gene of the SARS-CoV-2 isolate. Rename the file comp90016_COVID19_N.fasta.

Download the file comp90016_9b.fasta file from the LMS and place it in the same directory as the comp90016_COVID19_N.fasta file.

This file contains ORF-9b sequences from a selection of characterised coronaviruses.

BLAST is already installed on the COMP90016 server. If you would like to use your personal computer, installation instructions can be found here https://www.ncbi.nlm.nih.gov/books/NBK279671/.


Navigate to the directory containing the fasta files. 

First, we need to build a custom BLAST database from the ORF-9b sequences. Do this with the following command.

`makeblastdb -dbtype nucl -in comp90016_assignment_2_9b.fasta`

You will see some extra files have appeared.

Next, we will perform the BLAST search with the following command.

`blastn -query comp90016_COVID19_N.fasta -db comp90016_assignment_2_9b.fasta -outfmt "7 std ppos"`

The results will be printed in tabulated form. You can pipe the results to a file if you prefer.

### Questions

In the cells below, complete the following tasks:

2.1. (5 marks, 50 words max) Explain one factor that is important to consider when selecting a database for a BLAST search.

2.2. (10 marks, 50 words max) Is there evidence from the BLAST results that the COVID-19 N gene has a internal ORF homologous to ORF-9b sequences in other coronaviruses? What was the highest scoring hit? 

2.3. (10 marks, 50 words max) Interpret the E-value of the top scoring BLAST hit.

2.4. (10 marks, 50 words max) Explain the difference between the “% identity” and “% positive” fields.

2.5. (10 marks, 50 words max) Modify the provided BLAST command to work with an input of a protein sequence FASTA file (named comp90016_COVID19_N_protein.fasta). Decrease the word-size parameter from the default value to 2. Please use the same custom database used previously. Note that you are not required to execute this command.

2.6. (10 marks, 50 words max) What effect would decreasing the word size have on the sensitivity (ability to detect true homologs) and run-time of the search?
    

~~ GRADED CELL (5 marks) - complete this cell ~~

Explain one factor that is important to consider when selecting a database for a BLAST search. **50 words max**

Database curation is one factor that might influence the choice of database for a BLAST search. Curated databases reduce redundancy and contain only up-to-date sequence information.

~~ GRADED CELL (10 marks) - complete this cell ~~

Is there evidence from the BLAST results that the COVID-19 N gene has a internal ORF homologous to ORF-9b sequences in other coronaviruses? What was the highest scoring hit? **50 words max**

The BLAST results provide evidence that the COVID-19 virus has a sequence homologous to ORF-9b sequences from other coronaviruses due to the E-values from the six hits being close to zero. The highest scoring hit was from bat coronavirus HKU3 ORF 9b, with a bit-score of 366.

~~ GRADED CELL (10 marks) - complete this cell ~~

Interpret the E-value of the top scoring BLAST hit. **50 words max**

There are 7.49e-105 hits predicted to occur between the query sequence and a random database of the same size, with equivalent or higher bit-score, using the same BLAST parameters.

~~ GRADED CELL (10 marks) - complete this cell ~~

Explain the difference between the “% identity” and “% positive” fields. **50 words max**

% identity is the percentage of positions in the alignment that are identical between query and database sequence. % positive is the percentage of alignment positions that have a positive score in a given substitution matrix.

~~ GRADED CELL (10 marks) - complete this cell ~~

Modify the provided BLAST command to work with an input of a protein sequence FASTA file (named comp90016_COVID19_N_protein.fasta). Decrease the word-size parameter from the default value to 2. Please use the same custom database used previously. Note that you are not required to execute this command. Write the command below. **50 words max**

tblastn -query comp90016_COVID19_N_protein.fasta -db comp90016_assignment_2_9b.fasta -word_size 2 -outfmt "7 std ppos" 

~~ GRADED CELL (10 marks) - complete this cell ~~

What effect would decreasing the word size have on the sensitivity (ability to detect true homologs) and run-time of the search? **50 words max**

Decreasing the word size results in a higher sensitivity to detect true homologs. True homologs are less likely to be missed due to mismatches with the seed. The greater number of seed extensions results in an increased run-time.